#COMP9312_Project_Q1: Shortest Distance Query

For more information about the project, please refer to the [project specification](https://cgi.cse.unsw.edu.au/~cs9312/24T2/project/). You can edit this file and add anything you like. We will only use the code cell of the `ShortestDistanceQuery` class for testing. You can add descriptions and some theoretical analysis (e.g. index space, query time complexity, and index time complexity) to this file without creating a separate PDF document.

**Note**: Make sure to **sequentially run all cells in each section** in order to carry over intermediate variables/packages to the next cell.

## 1. Code Template
You need to implement the `ShortestDistanceQuery` class to support shortest distance queries for large graphs (e.g. hundreds of millions of vertices and edges). A code template is given below.

The `ShortestDistanceQuery` class is initialized by a graph `G`. The data structure of `G` will be presented in the next section. The class calls the function `preprocess` to precompute some index structure for `G`. The `query` function has two inputs: `vertex_u` and `vertex_v`. The function outputs the shortest distance between `vertex_u` and `vertex_v` in `G`.

### Floyd algorithm

In [ ]:
class ShortestDistanceQuery(object):
    def __init__(self, G):
        self.G = G
        self.distance_matrix = [[-1] * self.G.vertex_num for _ in range(self.G.vertex_num)]
        self.preprocess(G)

    def preprocess(self, G):
        for i in range(G.vertex_num):
            self.distance_matrix[i][i] = 0
            for j in G.adj_list[i]:
                self.distance_matrix[i][j] = 1

        for k in range(G.vertex_num):
            for i in range(G.vertex_num):
                for j in range(G.vertex_num):
                    if self.distance_matrix[i][k] != -1 and self.distance_matrix[k][j] != -1:
                        if self.distance_matrix[i][j] == -1 or self.distance_matrix[i][j] > self.distance_matrix[i][k] + self.distance_matrix[k][j]:
                            self.distance_matrix[i][j] = self.distance_matrix[i][k] + self.distance_matrix[k][j]
        
    def query(self, vertex_u, vertex_v):
        return self.distance_matrix[vertex_u][vertex_v]

### BFS algorithm

In [ ]:
from collections import deque

class ShortestDistanceQuery(object):
    def __init__(self, G):
        self.G = G
        self.distance_matrix = [[-1] * self.G.vertex_num for _ in range(self.G.vertex_num)]
        self.preprocess(G)

    def preprocess(self, G):
        for i in range(G.vertex_num):
            self.bfs(i)
        
    def bfs(self, start):
        queue = deque([start])
        self.distance_matrix[start][start] = 0
        visited = [False] * self.G.vertex_num
        visited[start] = True
        
        while queue:
            u = queue.popleft()
            for v in self.G.adj_list[u]:
                if not visited[v]:
                    visited[v] = True
                    self.distance_matrix[start][v] = self.distance_matrix[start][u] + 1
                    queue.append(v)

    def query(self, vertex_u, vertex_v):
        return self.distance_matrix[vertex_u][vertex_v]


In [ ]:
from collections import deque

class ShortestDistanceQuery(object):
    def __init__(self, G):
        self.G = G

    def query(self, vertex_u, vertex_v):
        if vertex_u == vertex_v:
            return 0
        
        queue = deque([vertex_u])
        distances = [-1] * self.G.vertex_num
        distances[vertex_u] = 0
        
        while queue:
            current = queue.popleft()
            current_distance = distances[current]
            
            for neighbor in self.G.adj_list[current]:
                if distances[neighbor] == -1:  # If the neighbor hasn't been visited yet
                    distances[neighbor] = current_distance + 1
                    queue.append(neighbor)
                    if neighbor == vertex_v:
                        return distances[neighbor]
        
        return -1  # If there is no path from vertex_u to vertex_v

### minimal 2-Hop algorithm

In [ ]:
from collections import defaultdict, deque
import heapq


class ShortestDistanceQuery:
    def __init__(self, G):
        self.G = G
        self.out_labels = defaultdict(dict)  # Out labels for each vertex
        self.in_labels = defaultdict(dict)   # In labels for each vertex
        self.preprocess(G)

    def preprocess(self, G):
        for v in range(G.vertex_num):
            self.bfs_label(v)

    def bfs_label(self, start):
        queue = deque([start])
        distances = {start: 0}
        self.out_labels[start][start] = 0
        self.in_labels[start][start] = 0

        while queue:
            current = queue.popleft()
            current_distance = distances[current]

            for neighbor in self.G.adj_list[current]:
                if neighbor not in distances:
                    distances[neighbor] = current_distance + 1
                    queue.append(neighbor)
                    if neighbor not in self.out_labels[start]:
                        self.out_labels[start][neighbor] = distances[neighbor]
                    if start not in self.in_labels[neighbor]:
                        self.in_labels[neighbor][start] = distances[neighbor]

    def query(self, vertex_u, vertex_v):
        if vertex_u == vertex_v:
            return 0

        min_distance = float('inf')
        out_labels_u = self.out_labels[vertex_u]
        in_labels_v = self.in_labels[vertex_v]

        common_vertices = set(out_labels_u.keys()).intersection(in_labels_v.keys())
        
        for intermediate in common_vertices:
            distance = out_labels_u[intermediate] + in_labels_v[intermediate]
            if distance < min_distance:
                min_distance = distance

        return min_distance if min_distance != float('inf') else -1

## 2. Graph Data Structure
The following is the data stucture of the input graph `G`.

In [ ]:
################################################################################
# Do not edit this code cell.
################################################################################

class UndirectedUnweightedGraph(object):
    def __init__(self, edge_list):
        self.adj_list = []
        self.vertex_num = 0
        self.edge_num = 0
        info = True
        for [vertex_u, vertex_v] in edge_list:
            if info:
                info = False
                self.vertex_num = vertex_u
                self.edge_num = vertex_v
                self.adj_list = [list() for _ in range(self.vertex_num)]
            else:
                self.adj_list[vertex_u].append(vertex_v)
                self.adj_list[vertex_v].append(vertex_u)

## 3. How to test your code

### 3.1 Download the sample dataset.

Running the following command will create the **COMP9312-24T2-Project** folder, which contains the files for the three datasets.

> **Cora** (2k vertices) is a real citation graph, **map_BJ_part** (4k vertices) is a real road network for a small area of Beijing, and **map_NY_part** (7k vertices) is a real road network for a small area of New York. For the two road networks, we erase the weight of each edge in the original dataset to generate unweighted graphs for simplicity.

There are three files for each dataset, where ***.graph** includes the graph information and all graph edges (vertex IDs are consecutive and start from 0), ***.query** includes a set of queries for testing. ***.answer** includes the correct answer to each query for your reference.

If the dataset already exists, an error like "*destination path 'COMP9312-24T2-Project' already exists*" will appear.

**NOTE**: We will test the code using different datasets.

In [ ]:
!git clone https://github.com/kevinChnn/COMP9312-24T2-Project

### 3.2 The main function

Our test procedure first loads the graph dataset and the query dataset. Then, it calls the `ShortestDistanceQuery` class to preprocess the graph. After that, it will run each query and test its efficiency and correctness.

In [ ]:
import time
import numpy as np

if __name__ == "__main__":

    print('\n######## Loading the dataset...')
    edge_list = np.loadtxt('./COMP9312-24T2-Project/cora.graph', dtype=int)
    query_list = np.loadtxt('./COMP9312-24T2-Project/cora-sd.query', dtype=int)
    # edge_list = np.loadtxt('./COMP9312-24T2-Project/map_BJ_part.graph', dtype=int)
    # query_list = np.loadtxt('./COMP9312-24T2-Project/map_BJ_part-sd.query', dtype=int)
    # edge_list = np.loadtxt('./COMP9312-24T2-Project/map_NY_part.graph', dtype=int)
    # query_list = np.loadtxt('./COMP9312-24T2-Project/map_NY_part-sd.query', dtype=int)
    G = UndirectedUnweightedGraph(edge_list)

    print('\n######## Preprocessing the graph...')
    start_preprocessing = time.time()
    SDQ = ShortestDistanceQuery(G)
    end_preprocessing = time.time()
    print("Preprocessing time: {:.8f}".format(end_preprocessing-start_preprocessing))

    print('\n######## Query Testing...')
    start_query = time.time()
    for i in range(query_list.shape[0]):
        distance = SDQ.query(query_list[i][0], query_list[i][1])
        print("The shortest distance between {} and {} is: {}".format(query_list[i][0], query_list[i][1], query_list[i][2]==distance))
    end_query = time.time()
    print("Average query time: {:.8f}".format((end_query-start_query)/query_list.shape[0]))